# INICIO DE ALGORITMO PARA RECOMENDACIÓN POR HABITOS DE **CONSUMIDOR**


In [12]:
# Algoritmo de consumo Market Basket Analyst.
# Consumo E-Commerce
# TPE
# CatoXP

import sqlite3 # importamos el sqlite3
import pandas as pd # libreria para manipulación de datos

In [13]:
# Primero se conecta a la base de datos SQLite
conexion = sqlite3.connect("DBsanoyfresco.db")

In [14]:
# Cargar una tabla completa en un DataFrame
df = pd.read_sql_query("SELECT * FROM tickets", conexion)

In [15]:
# Mostrar los primeros registros del DataFrame
df

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,1,112108,2023-02-02 00:00:00,10,4,83,49683,Pepino Kirby,0.99,4,3.96
1,1,112108,2023-02-02 00:00:00,10,4,24,13176,Bolsa de Bananas Orgánicas,2.45,4,9.80
2,1,112108,2023-02-02 00:00:00,10,4,24,47209,Aguacate Hass Orgánico,1.79,2,3.58
3,2,202279,2023-02-03 00:00:00,9,4,83,28985,Col Rizada Orgánica de Michigan,2.55,2,5.10
4,2,202279,2023-02-03 00:00:00,9,4,83,17794,Zanahorias,0.95,5,4.75
...,...,...,...,...,...,...,...,...,...,...,...
4975713,3421080,52726,2023-01-09 00:00:00,11,16,84,27845,Leche Entera Orgánica,0.99,3,2.97
4975714,3421080,52726,2023-01-09 00:00:00,11,4,83,41950,Racimo de Tomates Orgánicos,2.25,7,15.75
4975715,3421080,52726,2023-01-09 00:00:00,11,4,16,31717,Cilantro Orgánico,0.69,5,3.45
4975716,3421082,175185,2023-08-22 00:00:00,18,4,24,16797,Fresas,2.05,4,8.20


In [39]:
# verificamos la calidad de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id_pedido        int64         
 1   id_cliente       int64         
 2   fecha            datetime64[ns]
 3   hora             int64         
 4   id_departamento  int64         
 5   id_seccion       int64         
 6   id_producto      int64         
 7   nombre_producto  object        
 8   precio_unitario  float64       
 9   cantidad         int64         
 10  precio_total     float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 417.6+ MB


In [17]:
# la columna fecha se encuentra en Dtype objeto, se tuvo que cambiar a datetime


df["fecha"] = pd.to_datetime(df["fecha"])

In [18]:
# para el algoritmo no necesito todo el dataframe solamente el pedido y el nombre del producto
# asi lo llamaré como df_cesta = dataframe cesta

df_cesta = df[["id_pedido","nombre_producto"]]
df_cesta

,id_pedido,nombre_producto
0,1,Pepino Kirby
1,1,Bolsa de Bananas Orgánicas
2,1,Aguacate Hass Orgánico
3,2,Col Rizada Orgánica de Michigan
4,2,Zanahorias
...,...,...
4975713,3421080,Leche Entera Orgánica
4975714,3421080,Racimo de Tomates Orgánicos
4975715,3421080,Cilantro Orgánico
4975716,3421082,Fresas


In [19]:
# agrupamos los productos por id_pedido con groupby
df_agrupado = df_cesta.groupby("id_pedido")["nombre_producto"].apply(lambda producto: ",".join(producto))
df_agrupado

,nombre_producto
id_pedido,
1,"Pepino Kirby,Bolsa de Bananas Orgánicas,Aguaca..."
2,"Col Rizada Orgánica de Michigan,Zanahorias"
3,Espinacas Baby Orgánicas
5,"Bolsa de Bananas Orgánicas,Frambuesas Orgánica..."
10,"Banana,Cilantro Orgánico,Aguacate Orgánico,Ceb..."
...,...
3421077,"Frambuesas Orgánicas,Calabacín Orgánico,Leche ..."
3421078,"Manzanas Gala Orgánicas,Banana"
3421080,"Leche Entera Orgánica,Racimo de Tomates Orgáni..."


In [20]:
# aplicamos una funcion pandas .get_dummies() para transformar
# productos en columnas con 0 / 1
# si fue comprado en ese pedido tiene un 1 de lo contrario tiene un 0
df_transacciones = df_agrupado.str.get_dummies(sep=",")
df_transacciones

,Agua con Gas de Pomelo,Aguacate Hass Orgánico,Aguacate Orgánico,Ajo Orgánico,Apio Orgánico en Ramillete Pequeño,Arándanos Orgánicos,Banana,Bolsa de Bananas Orgánicas,Calabacín Orgánico,Cebolla Amarilla Orgánica,...,Manzana Honeycrisp Orgánica,Manzanas Gala Orgánicas,Pepino Kirby,Pepino Orgánico,Racimo de Tomates Orgánicos,Rúcula Baby Orgánica,Tomates Cherry Orgánicos,Uvas Rojas sin Semillas,Zanahorias,Zanahorias Baby Orgánicas
id_pedido,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421077,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3421078,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3421080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
# soporte para cada producto, al ser solamente 1 y 0 podemos realizar una media * 100
# para poder realizar el algoritmo mátematico p(aUb)
soporte = df_transacciones.mean() * 100
soporte.sort_values(ascending=False)

,0
Banana,23.809381
Bolsa de Bananas Orgánicas,19.129710
Fresas Orgánicas,13.349170
Espinacas Baby Orgánicas,12.198935
Aguacate Hass Orgánico,10.703004
Aguacate Orgánico,8.929331
Limón Grande,7.791279
Fresas,7.241960
Limones,7.107944
Leche Entera Orgánica,6.918301


In [41]:
# Aplicamos lógica con funciones para calcular la confianza entre dos productos en la muestra
def confianza(antecedente, consecuente):
  # casos donde se compran ambos productos -
  conjunto_ac = df_transacciones[(df_transacciones[antecedente] == 1) &  # Filtra solo las compras que tienen A == 1 y B == 1
                                 (df_transacciones[consecuente] == 1)]
  # Confianza - compras conjuntas / compras de producto A
  return len(conjunto_ac) / df_transacciones[antecedente].sum()   # Fórmula: Compras con ambos / Compras con A

"""
probabilidad de la intersección de A y B sería igual a
P (aUb) = FAB / FC frecuencia de A y frecuencia de B entre frecuencia A
en esa compra debe de ser == 1 antecedentes = 1 y consecuente == 1.
Ya que las compras se componen de 1.
"""
# Funcion para calcular el lift entre dos productos en la muestra
def lift(antecedente, consecuente):
  soporte_a = df_transacciones[antecedente].mean() # P(A)
  soporte_c = df_transacciones[consecuente].mean() # P(B)
  conteo_ac = len(df_transacciones[(df_transacciones[antecedente] == 1) &
                                    (df_transacciones[consecuente] == 1)]) # número de transacciones con A y B
  soporte_ac = conteo_ac / len(df_transacciones) # P(A ∩ B)
  return soporte_ac / (soporte_a * soporte_c) # P(A ∩ B) / (P(A) * P(B))



In [42]:
from itertools import combinations
# definir un umbral para la confianza minima
umbral_confianza = 0.05
asociaciones = []

# generar combinaciones de productos y calcular confianza y lift
for antecedente, consecuente in combinations(df_transacciones.columns,2):
  # soporte del antecedente
  soporte_a = df_transacciones[antecedente].mean()

  #calcular confiaza
  conf = confianza(antecedente, consecuente)
  if conf > umbral_confianza:
    asociaciones.append({
        'antecedente': antecedente,
        'consecuente': consecuente,
        'soporte_a' : round(soporte_a *100,1),
        'confianza' : round(conf * 100,1),
        'lift': round(lift(antecedente, consecuente),1)
    })

# convertir las asociaciones en un DataFrame
df_asociaciones = pd.DataFrame(asociaciones)

#ordenar las asociaciones por confianza de mayor a menor
df_asociaciones.sort_values(by='lift', ascending=False, inplace=True)
df_asociaciones # show

,antecedente,consecuente,soporte_a,confianza,lift
185,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6
224,Cilantro Orgánico,Limones,3.5,25.4,3.6
61,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5
106,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3
102,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1
...,...,...,...,...,...
136,Bolsa de Bananas Orgánicas,Limón Grande,19.1,5.2,0.7
38,Aguacate Orgánico,Bolsa de Bananas Orgánicas,8.9,13.0,0.7
58,Ajo Orgánico,Banana,5.5,17.4,0.7
3,Agua con Gas de Pomelo,Bolsa de Bananas Orgánicas,3.8,13.2,0.7


In [43]:
# crear una tabla con los productos únicos y las columnas correspondientes
productos_unicos = df[['id_producto','id_seccion','id_departamento','nombre_producto']].drop_duplicates()
productos_unicos

,id_producto,id_seccion,id_departamento,nombre_producto
0,49683,83,4,Pepino Kirby
1,13176,24,4,Bolsa de Bananas Orgánicas
2,47209,24,4,Aguacate Hass Orgánico
3,28985,83,4,Col Rizada Orgánica de Michigan
4,17794,83,4,Zanahorias
5,21903,123,4,Espinacas Baby Orgánicas
7,27966,123,4,Frambuesas Orgánicas
9,24852,24,4,Banana
10,31717,16,4,Cilantro Orgánico
11,47766,24,4,Aguacate Orgánico


In [44]:
df_asociaciones_enriquecido = df_asociaciones.merge(productos_unicos, left_on='antecedente', right_on='nombre_producto', how='left').drop(columns=['nombre_producto'])
df_asociaciones_enriquecido.columns = ['antecedente', 'consecuente', 'soporte_a', 'confianza', 'lift', 'id_producto_a', 'id_seccion_a', 'id_departamento_a']
df_asociaciones_enriquecido

,antecedente,consecuente,soporte_a,confianza,lift,id_producto_a,id_seccion_a,id_departamento_a
0,Cebolla Roja Orgánica,Cilantro Orgánico,3.4,12.8,3.6,8518,83,4
1,Cilantro Orgánico,Limones,3.5,25.4,3.6,31717,16,4
2,Ajo Orgánico,Cebolla Amarilla Orgánica,5.5,20.1,3.5,24964,83,4
3,Apio Orgánico en Ramillete Pequeño,Zanahorias,3.4,12.0,3.3,44359,83,4
4,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,3.4,12.7,3.1,44359,83,4
...,...,...,...,...,...,...,...,...
394,Bolsa de Bananas Orgánicas,Limón Grande,19.1,5.2,0.7,13176,24,4
395,Aguacate Orgánico,Bolsa de Bananas Orgánicas,8.9,13.0,0.7,47766,24,4
396,Ajo Orgánico,Banana,5.5,17.4,0.7,24964,83,4
397,Agua con Gas de Pomelo,Bolsa de Bananas Orgánicas,3.8,13.2,0.7,44632,115,7


In [46]:
df_asociaciones_enriquecido.to_csv("reglas.csv", index=False,sep=";",decimal=",")